In [1]:
import numpy as np
import pandas as pd

In [2]:
train_fn = '../out/bighead/oxford_comma/head_train_scored.csv'
valid_fn = '../out/bighead/oxford_comma/head_test_scored.csv'
# train_fn = '../out/bighead/zero_that/train_scored.csv'
# valid_fn = '../out/bighead/zero_that/test_scored.csv'
toi = 11    # ,
# toi = 326    # that

In [3]:
train_df = pd.read_csv(train_fn, names=['line', 'prefix_tok_len', 'contains', 'probability', 'argmax'])
valid_df = pd.read_csv(valid_fn, names=['line', 'prefix_tok_len', 'contains', 'probability', 'argmax'])
train_df.head(1)

,line,prefix_tok_len,contains,probability,argmax
0,14,144,True,0.00028,290


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-j-6B', truncation_side='left')

In [5]:
tokenizer.decode([11, 290, 50256, 13, 5523, 544, 544])

', and<|endoftext|>.Âiaia'

In [6]:
tokenizer.decode([340, 326, 314, 262, 345, 356])

' it that I the you we'

In [7]:
train_df['argmax_is_comma'] = train_df.argmax == toi
valid_df['argmax_is_comma'] = valid_df.argmax == toi

In [8]:
subset_train_df = train_df[train_df.prefix_tok_len > 50]
subset_valid_df = valid_df[valid_df.prefix_tok_len > 50]
len(train_df), len(valid_df)

(9999, 9999)

# Summary statistics

In [9]:
subset_train_df.argmax.value_counts()[:10]

11      6542
290     2243
393      273
13        34
286       19
8         13
220        9
1222       6
447        6
82         6
Name: argmax, dtype: int64

In [10]:
subset_train_df.contains.value_counts(normalize=True)

False    0.578264
True     0.421736
Name: contains, dtype: float64

In [11]:
subset_valid_df.contains.value_counts(normalize=True)

False    0.575441
True     0.424559
Name: contains, dtype: float64

# Analysis

In [12]:
correctness = subset_train_df.argmax_is_comma == subset_train_df.contains
np.mean(correctness), np.std(correctness)/np.sqrt(len(correctness))

(0.6386776510350655, 0.004936954759415384)

In [13]:
correctness = subset_valid_df.argmax_is_comma == subset_valid_df.contains
np.mean(correctness), np.std(correctness)/np.sqrt(len(correctness))

(0.6417484954070319, 0.004926954172765392)

In [14]:
subset_train_df.groupby('contains').mean()

,line,prefix_tok_len,probability,argmax,argmax_is_comma
contains,,,,,
False,46179.751963,226.583014,0.000315,478.894612,0.545205
True,46098.384924,233.709742,0.000270,146.695467,0.890809


In [15]:
subset_valid_df.groupby('contains').mean()

,line,prefix_tok_len,probability,argmax,argmax_is_comma
contains,,,,,
False,45069.711193,228.264587,0.000465,464.102936,0.544404
True,44602.764735,236.698334,0.000221,187.811738,0.894056


In [16]:
subset_train_df.groupby('contains').sem()

,line,prefix_tok_len,probability,argmax,argmax_is_comma
contains,,,,,
False,356.162880,1.289466,0.00003,35.652450,0.006730
True,416.611092,1.797477,0.00003,25.319212,0.004936
